In [9]:
%cd mmdetection

[Errno 2] No such file or directory: 'mmdetection'
/mnt/nas2/users/eslim/workspace/comp/mmdetection


In [10]:

import os
from natsort import natsorted
from mmdet.apis import DetInferencer
from PIL import Image
import json
from mmengine import ConfigDict
from mmengine.config import Config, DictAction
import copy


os.environ["CUDA_VISIBLE_DEVICES"]="2"

device = 'cuda:0'
config_path = "projects/CO-DETR/configs/codino/co_dino_5scale_r50_lsj_8xb2_1x_resnet101.py"
checkpoint = "./work_dirs/resnet101/refine/iter_118000.pth"
cfg = Config.fromfile(config_path)

image_path_prefix = "./data/all_dataset/roboflow_data/train/"
save_prefix = "./data/all_dataset/anno/"

extra_image_num = 500
pseudo_thres = 0.65

In [11]:
inferencer = DetInferencer(cfg, checkpoint, device)


Loads checkpoint by local backend from path: ./work_dirs/resnet101/refine/iter_118000.pth


/mnt/nas2/users/eslim/workspace/comp/mmdetection/projects/CO-DETR/codetr/transformer.py:1327: UserWarning: If you want to reduce GPU memory usage,                               please install fairscale by executing the                               following command: pip install fairscale.
  following command: pip install fairscale.')


09/17 01:30:09 - mmengine - INFO - 
rpn_conv.weight - torch.Size([256, 256, 3, 3]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/17 01:30:09 - mmengine - INFO - 
rpn_conv.bias - torch.Size([256]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/17 01:30:09 - mmengine - INFO - 
rpn_cls.weight - torch.Size([9, 256, 1, 1]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/17 01:30:09 - mmengine - INFO - 
rpn_cls.bias - torch.Size([9]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/17 01:30:09 - mmengine - INFO - 
rpn_reg.weight - torch.Size([36, 256, 1, 1]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/17 01:30:09 - mmengine - INFO - 
rpn_reg.bias - torch.Size([36]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/17 01:30:09 - mmengine - INFO - 
bbox_head.fc_cls.weight - torch.Size([9, 1024]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/17 01:30:09 - mmengine - INFO - 
bbox_head.fc_cls.bias - torch.Size([9]): 
NormalInit: mean=0, std=0.01, bias=0 
 
09/17 01:30:09 - mmengine - INFO - 
bbox_head.fc_reg.we

/mnt/nas2/users/eslim/workspace/comp/mmdetection/mmdet/models/dense_heads/anchor_head.py:108: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '
/opt/conda/lib/python3.7/site-packages/mmengine/visualization/visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '


In [12]:

with open('./data/all_dataset/roboflow_data/train/_annotations.coco.json', 'r') as file:
    ext_train_data = json.load(file)



In [13]:
len(ext_train_data)
ext_train_data["images"][-1]

{'id': 7500,
 'license': 1,
 'file_name': 'FLIR_08894_jpeg.rf.dd97a2a8830185002b78ff8d7b653681.jpg',
 'height': 640,
 'width': 640,
 'date_captured': '2024-09-03T13:23:20+00:00'}

In [14]:
ext_train_data.keys()


dict_keys(['info', 'licenses', 'categories', 'images', 'annotations'])

In [15]:
ext_train_data["annotations"]

[{'id': 0,
  'image_id': 0,
  'category_id': 2,
  'bbox': [178, 281, 25, 30],
  'area': 750,
  'segmentation': [[203,
    311.25,
    203,
    281.25,
    178,
    281.25,
    178,
    311.25,
    203,
    311.25]],
  'iscrowd': 0},
 {'id': 1,
  'image_id': 0,
  'category_id': 2,
  'bbox': [254, 276, 40, 18.75],
  'area': 750,
  'segmentation': [[294, 295, 294, 276.25, 254, 276.25, 254, 295, 294, 295]],
  'iscrowd': 0},
 {'id': 2,
  'image_id': 0,
  'category_id': 2,
  'bbox': [217, 276, 37, 22.5],
  'area': 832.5,
  'segmentation': [[254,
    298.75,
    254,
    276.25,
    217,
    276.25,
    217,
    298.75,
    254,
    298.75]],
  'iscrowd': 0},
 {'id': 3,
  'image_id': 1,
  'category_id': 1,
  'bbox': [148, 335, 60, 67.5],
  'area': 4050,
  'segmentation': [[208, 402.5, 208, 335, 148, 335, 148, 402.5, 208, 402.5]],
  'iscrowd': 0},
 {'id': 4,
  'image_id': 1,
  'category_id': 4,
  'bbox': [155, 283, 49, 113.75],
  'area': 5573.75,
  'segmentation': [[204,
    396.25,
    204,
 

In [16]:
total_num=0
ret_list=[]
for idx,info in enumerate(ext_train_data["annotations"]):
    file_name = ext_train_data["images"][info["image_id"]]["file_name"]
    image_path = os.path.join(image_path_prefix,file_name)
    results = inferencer(image_path)
    
    if total_num>extra_image_num:
        break
    total_num+=1
    for ret_idx,bbox in enumerate(results["predictions"][0]["bboxes"]):
        if results['predictions'][0]["scores"][ret_idx] > pseudo_thres:
            min_x, min_y, max_x, max_y = bbox
            ret = {
                    "image_id" : file_name,
                    "category_id" : results['predictions'][0]["labels"][ret_idx],
                    "bbox":[
                        min_x,
                        min_y,
                        max_x-min_x,
                        max_y-min_y
                    ],
                    "score":results['predictions'][0]["scores"][ret_idx],
                }
            ret_list.append(ret)
        
    


Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

Output()

In [17]:
with open(os.path.join(save_prefix,"extra_pseudo_ann.txt"), "w") as file:
    json.dump(ret_list, file, indent=4)